# PASSO 00 IMPORTS

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
import boto3
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


import requests
from datetime import datetime
import boto3
import json
import time


# PASSO 01 DATA DESCRIPTION

In [29]:
def get_historical_data(coin, days=90):
    url = f'https://api.coingecko.com/api/v3/coins/{coin}/market_chart?vs_currency=usd&days={days}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        prices = data['prices']
        df = pd.DataFrame(prices, columns=["Timestamp", "Price (USD)"])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
        return df
    except Exception as e:
        st.error(f"Erro ao buscar dados históricos: {e}")
        return pd.DataFrame()

In [30]:
historical_data = get_historical_data('bitcoin',days=30)
historical_data.to_parquet('../data/raw/historical_data.parquet')

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregar os dados (supondo que já tenha sido coletado)
df_principal = pd.read_parquet('../data/raw/historical_data.parquet')

# FEATURE ENGINEERING

In [32]:
df = df_principal.copy()
df.head()

,Timestamp,Price (USD)
0,2024-12-18 13:03:52.431,104983.481292
1,2024-12-18 14:07:46.894,104661.925495
2,2024-12-18 15:03:54.033,103821.249179
3,2024-12-18 16:01:16.381,104432.307228
4,2024-12-18 17:00:45.402,104094.339001


In [33]:
df = df.rename(columns={"Timestamp":"timestamp","Price (USD)":"price (usd)"})
df

,timestamp,price (usd)
0,2024-12-18 13:03:52.431,104983.481292
1,2024-12-18 14:07:46.894,104661.925495
2,2024-12-18 15:03:54.033,103821.249179
3,2024-12-18 16:01:16.381,104432.307228
4,2024-12-18 17:00:45.402,104094.339001
...,...,...
715,2025-01-17 09:08:35.792,102125.380533
716,2025-01-17 10:03:39.901,101924.678408
717,2025-01-17 11:03:33.572,102243.203797
718,2025-01-17 12:01:23.197,102254.354934


In [34]:
df['day_of_week'] = df['timestamp'].dt.dayofweek


# Extrair o número do dia no mês
df['day_of_month'] = df['timestamp'].dt.day

In [35]:
# Média móvel de 7 dias (curto prazo)
df['moving_avg_7'] = df['price (usd)'].rolling(window=7).mean()

# Média móvel de 30 dias (médio prazo)
df['moving_avg_30'] = df['price (usd)'].rolling(window=30).mean()

# Média móvel de 90 dias (longo prazo)
df['moving_avg_90'] = df['price (usd)'].rolling(window=90).mean()

In [36]:
# Calcular a variação percentual diária (retorno)
df['daily_return'] = df['price (usd)'].pct_change()

# Calcular a variação absoluta diária
df['daily_return_abs'] = df['price (usd)'].diff()

In [37]:
df['volatility'] = df['daily_return'].rolling(window=30).std()

In [38]:
# Função para calcular o RSI
def calculate_rsi(data, window):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calcular o RSI de 14 dias
df['RSI_14'] = calculate_rsi(df['price (usd)'], window=14)

In [39]:
# Média exponencial de 7 dias
df['ema_7'] = df['price (usd)'].ewm(span=7, adjust=False).mean()

# Média exponencial de 30 dias
df['ema_30'] = df['price (usd)'].ewm(span=30, adjust=False).mean()

# Variable Filtering

In [40]:
df = df.dropna()

In [41]:
# Lista das variáveis derivadas
derived_columns = [
    'day_of_week',  'day_of_month', 
    'moving_avg_7', 'moving_avg_30', 'moving_avg_90'
]

# Filtrando apenas as variáveis derivadas
df_derived = df[derived_columns]

# Exibindo as primeiras linhas das variáveis derivadas
print(df_derived.head())

    day_of_week  day_of_month  moving_avg_7  moving_avg_30  moving_avg_90
89            6            22  96760.174188   97473.094347   98655.367341
90            6            22  96680.409143   97445.699704   98563.173131
91            6            22  96534.748756   97405.204670   98468.629990
92            6            22  96470.464219   97378.396312   98387.899618
93            6            22  96508.849141   97361.053188   98304.120821


In [42]:
scaler = StandardScaler()
# Separar variáveis independentes (apenas as derivadas)
X_derived = df_derived

# Variável dependente (preço)
y = df['price (usd)']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_derived, y, test_size=0.3, shuffle=False)

# Normalizar as variáveis independentes
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
# Inicializar e treinar o modelo de regressão linear
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Prever no conjunto de teste
y_pred = linear_model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (Random Forest): {mse}')
print(f'Root Mean Squared Error (RMSE) (Random Forest): {mse}')
print(f'Mean Absolute Error (MAE) (Random Forest): {mae}')
print(f'R² (Random Forest): {r2}')

Mean Squared Error (Random Forest): 380683.3870979732
Root Mean Squared Error (RMSE) (Random Forest): 380683.3870979732
Mean Absolute Error (MAE) (Random Forest): 464.16382092243003
R² (Random Forest): 0.9500704896683985


In [44]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

# ------------------ Treinamento com XGBoost ------------------
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.4, random_state=42)
xgb_model.fit(X_train, y_train)

# Previsão com XGBoost
xgb_y_pred = xgb_model.predict(X_test)

# Avaliação do XGBoost
xgb_mse = mean_squared_error(y_test, xgb_y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mae = mean_absolute_error(y_test, xgb_y_pred)
xgb_r2 = r2_score(y_test, xgb_y_pred)

print("------ XGBoost Results ------")
print(f'Mean Squared Error (XGBoost): {xgb_mse}')
print(f'Root Mean Squared Error (RMSE) (XGBoost): {xgb_rmse}')
print(f'Mean Absolute Error (MAE) (XGBoost): {xgb_mae}')
print(f'R² (XGBoost): {xgb_r2}')


# ------------------ Treinamento com Gradient Boosting ------------------
gb_model = GradientBoostingRegressor(n_estimators=10, learning_rate=0.01, random_state=42)
gb_model.fit(X_train, y_train)

# Previsão com Gradient Boosting
gb_y_pred = gb_model.predict(X_test)

# Avaliação do Gradient Boosting
gb_mse = mean_squared_error(y_test, gb_y_pred)
gb_rmse = np.sqrt(gb_mse)
gb_mae = mean_absolute_error(y_test, gb_y_pred)
gb_r2 = r2_score(y_test, gb_y_pred)

print("\n------ Gradient Boosting Results ------")
print(f'Mean Squared Error (Gradient Boosting): {gb_mse}')
print(f'Root Mean Squared Error (RMSE) (Gradient Boosting): {gb_rmse}')
print(f'Mean Absolute Error (MAE) (Gradient Boosting): {gb_mae}')
print(f'R² (Gradient Boosting): {gb_r2}')

------ XGBoost Results ------
Mean Squared Error (XGBoost): 1063933.1473117762
Root Mean Squared Error (RMSE) (XGBoost): 1031.4713506984942
Mean Absolute Error (MAE) (XGBoost): 813.6469017215838
R² (XGBoost): 0.8604571072150169

------ Gradient Boosting Results ------
Mean Squared Error (Gradient Boosting): 6311664.69069127
Root Mean Squared Error (RMSE) (Gradient Boosting): 2512.302667015117
Mean Absolute Error (MAE) (Gradient Boosting): 2137.9348077680534
R² (Gradient Boosting): 0.17217735770967546


In [46]:
import joblib
import os

# Caminho local para salvar o modelo
local_model_path = '../models/rl_model.joblib'


# Salvar o modelo
joblib.dump(linear_model, local_model_path)

print(f"Modelo salvo localmente em: {local_model_path}")


Modelo salvo localmente em: ../models/rl_model.joblib


In [143]:
import boto3

class S3Services:
    def __init__(self, bucket_name):
        self.s3_client = boto3.client('s3')
        self.bucket_name = bucket_name

    def upload_file(self, file_name, s3_key):
        """
        Faz upload de um arquivo para um bucket S3.
        :param file_name: Caminho local do arquivo.
        :param s3_key: Caminho no S3 onde o arquivo será salvo.
        """
        try:
            self.s3_client.upload_file(file_name, self.bucket_name, s3_key)
            print(f"Arquivo {file_name} enviado com sucesso para {s3_key}.")
        except Exception as e:
            print(f"Erro ao enviar o arquivo para o S3: {str(e)}")

In [144]:
# Inicializar o serviço S3
bucket_name = 'my-firehose-bucket'  # Substitua pelo nome do seu bucket S3
s3_service = S3Services(bucket_name)

# Caminho no S3 onde o arquivo será salvo
s3_model_path = 'ml_models/rl_model.pkl'

# Upload do modelo para o S3
s3_service.upload_file(local_model_path, s3_model_path)

Arquivo ../models/rl_model.pkl enviado com sucesso para ml_models/rl_model.pkl.


In [145]:
X_train.columns

Index(['day_of_week', 'day_of_month', 'moving_avg_7', 'moving_avg_30',
       'moving_avg_90'],
      dtype='object')